In [1]:
'''import tensorflow as tf
import numpy as np
import pickle
tf.set_random_seed(10)

# 초성 리스트. 00 ~ 18 --> 19개
CHOSUNG_LIST = ['ᄀ', 'ᄁ', 'ᄂ', 'ᄃ', 'ᄄ', 'ᄅ', 'ᄆ', 'ᄇ', 'ᄈ', 'ᄉ', 'ᄊ', 'ᄋ', 'ᄌ', 'ᄍ', 'ᄎ', 'ᄏ', 'ᄐ', 'ᄑ', 'ᄒ']

# 중성 리스트. 00 ~ 20 --> 21개
JUNGSUNG_LIST = ['ᅡ', 'ᅢ', 'ᅣ', 'ᅤ', 'ᅥ', 'ᅦ', 'ᅧ', 'ᅨ', 'ᅩ', 'ᅪ', 'ᅫ', 'ᅬ', 'ᅭ', 'ᅮ', 'ᅯ', 'ᅰ', 'ᅱ', 'ᅲ', 'ᅳ', 'ᅴ',
                 'ᅵ']

# 종성 리스트. 00 ~ 27 + 1(1개는 종성없음코드) --> 28개
JONGSUNG_LIST = [' ', 'ᆨ', 'ᆩ', 'ᆪ', 'ᆫ', 'ᆬ', 'ᆭ', 'ᆮ', 'ᆯ', 'ᆰ', 'ᆱ', 'ᆲ', 'ᆳ', 'ᆴ', 'ᆵ', 'ᆶ', 'ᆷ', 'ᆸ', 'ᆹ', 'ᆺ',
                 'ᆻ', 'ᆼ', 'ᆽ', 'ᆾ', 'ᆿ', 'ᇀ', 'ᇁ', 'ᇂ']

# 독립 자소 리스트. --> 51개
INDI_LIST = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅄ', 'ㅅ',
             'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ',
             'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

def syllable(char):
    s = ord(char) - 44032
    cho = (s//21)//28
    jung = (s%(21*28))//28
    jong = (s%28)
    
    return CHOSUNG_LIST[cho], JUNGSUNG_LIST[jung], JONGSUNG_LIST[jong]

# 자소의 차원은 119
JASO_DIM = len(CHOSUNG_LIST)+len(JUNGSUNG_LIST)+len(JONGSUNG_LIST)+len(INDI_LIST)

hangul_johab = range(44032,55204)
hangul_jaeum = range(12593,12623)
hangul_moeum = range(12623,12644)
hangul_chosung = range(4352,4371)
hangul_jungsung = range(4449,4470)
hangul_jongsung = range(4520,4547)
english1 = range(65,91)
english2 = range(97,123)
digit = range(48,58)
special_char = [ord('.'), ord('\''), ord('?'), ord(','), ord('!'), ord('%')] # 형태소 분석에 필요하다고 생각하는 특수문자 추가

def read_data(file_path):
    sentence = []
    data = []
    label = []
    d_append = data.append
    with open(file_path,"r") as f:
        for line in f.readlines():
            if line != '\n':
                w = line.split('\t')
                label.append(w[1].replace('\n',''))
                word = []
                w_append = word.append
                w_extend = word.extend
                for c in w[0]:
                    sign_unk = 0
                    
                    if ord(c) in hangul_johab or ord(c) in hangul_chosung or \
                       ord(c) in hangul_jungsung or ord(c) in hangul_jongsung or \
                       ord(c) in hangul_jaeum or ord(c) in hangul_moeum or \
                       ord(c) in english1 or ord(c) in english2 or \
                       ord(c) in digit or ord(c) in special_char: pass
                    else: sign_unk = 1 # 지정된 한글, 영어, 숫자, 특수문자 이외에 전부 UNK태그 지정

                    if sign_unk == 1:
                        w_append('<UNK>')
                    else:
                        if ord(c) in hangul_johab: # 조합형 한글은 자모를 분리
                            jaso = syllable(c)
                            w_extend(jaso)
                        else:
                            w_append(c) # 한글자모, 영어, 숫자는 그대로
                sentence.append((word,w[1].replace('\n',''))) # ([분리된 형태소],태그) 형태로 저장
            else:
                if sentence != []:
                    d_append(sentence) # sentence마다 구분지어서 저장
                    sentence = []
    return data,label
print("\n==Read Data==\n")
data, label = read_data("data_v5_edit.txt") # data는 3차원 리스트로 
                                                       # 전체 데이터 -> 문장 -> 형태소 순으로 저장됨

pickle.dump(data, open('data.pkl','wb'))
pickle.dump(label, open('label.pkl','wb'))

sentence_max_length = 0
word_max_length = 0
for sen in data:
    if sentence_max_length < len(sen): sentence_max_length = len(sen)
    for word in sen:
        if word_max_length < len(word[0]): word_max_length = len(word[0])

char_list = ['<UNK>']+CHOSUNG_LIST+JUNGSUNG_LIST+JONGSUNG_LIST+INDI_LIST+[chr(i) for i in english1]\
               + [chr(i) for i in english2] + [chr(i) for i in digit] + [chr(i) for i in special_char]
dictionary_char = dict()

for i in char_list:
    dictionary_char[i] = len(dictionary_char)

label = sorted(label)

dictionary_label= dict()
for i in label:
    dictionary_label[i] = len(dictionary_label)

dictionary_char

# 위에서 정의된 dictionary에 따라 데이터를 index로 치환
def make_dataSet(data, dictionary_char):
    indexed_data = [] 
    d_append = indexed_data.append
    for sentence in data:
        sen = []
        lab = []
        s_append = sen.append
        l_append = lab.append
        for word in sentence:
            s_append(([dictionary_char[char] for char in word[0]], dictionary_label[word[1]]))
        d_append(sen)
    
    return indexed_data

indexed_data = make_dataSet(data,dictionary_char)

dictionary_label'''

'import tensorflow as tf\nimport numpy as np\nimport pickle\ntf.set_random_seed(10)\n\n# 초성 리스트. 00 ~ 18 --> 19개\nCHOSUNG_LIST = [\'ᄀ\', \'ᄁ\', \'ᄂ\', \'ᄃ\', \'ᄄ\', \'ᄅ\', \'ᄆ\', \'ᄇ\', \'ᄈ\', \'ᄉ\', \'ᄊ\', \'ᄋ\', \'ᄌ\', \'ᄍ\', \'ᄎ\', \'ᄏ\', \'ᄐ\', \'ᄑ\', \'ᄒ\']\n\n# 중성 리스트. 00 ~ 20 --> 21개\nJUNGSUNG_LIST = [\'ᅡ\', \'ᅢ\', \'ᅣ\', \'ᅤ\', \'ᅥ\', \'ᅦ\', \'ᅧ\', \'ᅨ\', \'ᅩ\', \'ᅪ\', \'ᅫ\', \'ᅬ\', \'ᅭ\', \'ᅮ\', \'ᅯ\', \'ᅰ\', \'ᅱ\', \'ᅲ\', \'ᅳ\', \'ᅴ\',\n                 \'ᅵ\']\n\n# 종성 리스트. 00 ~ 27 + 1(1개는 종성없음코드) --> 28개\nJONGSUNG_LIST = [\' \', \'ᆨ\', \'ᆩ\', \'ᆪ\', \'ᆫ\', \'ᆬ\', \'ᆭ\', \'ᆮ\', \'ᆯ\', \'ᆰ\', \'ᆱ\', \'ᆲ\', \'ᆳ\', \'ᆴ\', \'ᆵ\', \'ᆶ\', \'ᆷ\', \'ᆸ\', \'ᆹ\', \'ᆺ\',\n                 \'ᆻ\', \'ᆼ\', \'ᆽ\', \'ᆾ\', \'ᆿ\', \'ᇀ\', \'ᇁ\', \'ᇂ\']\n\n# 독립 자소 리스트. --> 51개\nINDI_LIST = [\'ㄱ\', \'ㄲ\', \'ㄳ\', \'ㄴ\', \'ㄵ\', \'ㄶ\', \'ㄷ\', \'ㄸ\', \'ㄹ\', \'ㄺ\', \'ㄻ\', \'ㄼ\', \'ㄽ\', \'ㄾ\', \'ㄿ\', \'ㅀ\', \'ㅁ\', \'ㅂ\', \'ㅃ\', \'ㅄ\', \'ㅅ\',\n             \'ㅆ\', \'ㅇ\', \'ㅈ\', \'ㅉ\', \'ㅊ\', \'ㅋ\', \'ㅌ\', 

In [2]:
# pickle.dump(indexed_data,open('indexed_data.pkl','wb'))
# pickle.dump(dictionary_char,open('char_dict.pkl','wb'))
# pickle.dump(dictionary_label,open('label_dict.pkl','wb'))

In [3]:
import tensorflow as tf
import numpy as np
#from preprocessing import Preprocessing
tf.set_random_seed(10)

In [4]:
char_dim = 100
word_max_length = 32
X = tf.placeholder(tf.float32, [1, word_max_length, char_dim, 1])

In [5]:
print(X)

Tensor("Placeholder:0", shape=(1, 32, 100, 1), dtype=float32)


In [6]:
filter_size = [3,5,7,9]
num_filters = 25
conved_layers = []
for f in filter_size:
    Y = tf.layers.conv2d(X, filters=num_filters, kernel_size=[f, char_dim], activation=tf.nn.relu, padding= 'VALID',
                              kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    Y = tf.layers.max_pooling2d(Y, pool_size=[word_max_length - f + 1, 1], strides=4, padding='VALID')
    conved_layers.append(Y)
concat_chars = tf.concat(conved_layers,3)
#out_word = tf.squeeze(concat_chars)
#out_word = tf.reshape(concat_chars,[-1,15,100])

In [7]:
conved_layers

[<tf.Tensor 'max_pooling2d/MaxPool:0' shape=(1, 1, 1, 25) dtype=float32>,
 <tf.Tensor 'max_pooling2d_1/MaxPool:0' shape=(1, 1, 1, 25) dtype=float32>,
 <tf.Tensor 'max_pooling2d_2/MaxPool:0' shape=(1, 1, 1, 25) dtype=float32>,
 <tf.Tensor 'max_pooling2d_3/MaxPool:0' shape=(1, 1, 1, 25) dtype=float32>]

In [8]:
concat_chars

<tf.Tensor 'concat:0' shape=(1, 1, 1, 100) dtype=float32>

In [9]:
'''def make_input (indexed_data, window_size = 7, max_length = 48, char_dim = 188):
    for sentence in indexed_data:
        for w_index in range(len(sentence)):
            sentence_input = np.zeros([window_size*2 +1, max_length, char_dim], dtype=np.float32)
            word_index = 6 - w_index
            for word in sentence:
                word_index = word_index + 1
                if word_index >= 15 or word_index<0 : continue
                
                char_index = 0
                if len(word[0]) % 2 == 0:
                    char_index = max_length//2 - len(word[0]) // 2
                else:
                    char_index = max_length//2 - (len(word[0]) // 2 + 1)
                
                for i, char in enumerate(word[0]):
                    sentence_input[word_index][int(i+char_index)][char] = 1
            sentence_input = np.reshape(sentence_input, [window_size*2 +1, max_length, char_dim,1])
            yield sentence_input

len(indexed_data)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    X_input = make_input(indexed_data)
    y_1 = sess.run(out_word,feed_dict={X:next(X_input)})

## 101번째 index부터 position embedding & binary feature
## 7번째 target vector는 binary feature가 1

y_1[0][7]'''

'def make_input (indexed_data, window_size = 7, max_length = 48, char_dim = 188):\n    for sentence in indexed_data:\n        for w_index in range(len(sentence)):\n            sentence_input = np.zeros([window_size*2 +1, max_length, char_dim], dtype=np.float32)\n            word_index = 6 - w_index\n            for word in sentence:\n                word_index = word_index + 1\n                if word_index >= 15 or word_index<0 : continue\n                \n                char_index = 0\n                if len(word[0]) % 2 == 0:\n                    char_index = max_length//2 - len(word[0]) // 2\n                else:\n                    char_index = max_length//2 - (len(word[0]) // 2 + 1)\n                \n                for i, char in enumerate(word[0]):\n                    sentence_input[word_index][int(i+char_index)][char] = 1\n            sentence_input = np.reshape(sentence_input, [window_size*2 +1, max_length, char_dim,1])\n            yield sentence_input\n\nlen(indexed_d